# December 2017: Advent of Code

## Common imports & library functions

In [1]:
from collections import defaultdict, namedtuple
import doctest
import heapq
import itertools
import math
import numpy as np
import re

# Cribbed from norvig@
def nth(iterable, n, default=None):
    "Returns the nth item of iterable, or a default value"
    return next(itertools.islice(iterable, n, None), default)

def np_impulse(shape, idx, value, dtype=np.int):
    data = np.zeros(shape, dtype=dtype)
    data[idx] = value
    return data

## Day 1: Inverse Captcha

In [ ]:
def solve_captcha(captcha, offset=1):
    """
    >>> solve_captcha('1122')
    3
    >>> solve_captcha('1111')
    4
    >>> solve_captcha('1234')
    0
    >>> solve_captcha('91212129')
    9
    >>> solve_captcha('1212', 2)
    6
    >>> solve_captcha('1221', 2)
    0
    >>> solve_captcha('123425', 3)
    4
    >>> solve_captcha('123123', 3)
    12
    >>> solve_captcha('12131415', 4)
    4
    """
    return sum(int(captcha[i]) for i in range(len(captcha))
               if captcha[i] == captcha[i-offset])

In [ ]:
# Run unit tests
doctest.testmod()

In [ ]:
# Final answer
with open('day1_captcha.txt') as f:
    captcha = f.read().strip()
    print('Part 1: ', solve_captcha(captcha))
    print('Part 2: ', solve_captcha(captcha, len(captcha)//2))

## Day 2: Corruption Checksum

In [ ]:
def minmax_sum(row):
    return max(row) - min(row)

def even_quotient(row):
    for i in range(len(row)):
        for j in range(len(row)):
            if i == j: continue
            if row[i] % row[j] == 0: return row[i] // row[j]

def solve_checksum(spreadsheet, row_checksum=minmax_sum):
    """
    >>> solve_checksum('1 1\\n2 2', minmax_sum)
    0
    >>> solve_checksum('40 41\\n1 0 3 9', minmax_sum)
    10
    >>> solve_checksum('5 1 9 5\\n7 5 3\\n2 4 6 8', minmax_sum)
    18
    >>> solve_checksum('5 5\\n2 2\\n3 3', even_quotient)
    3
    >>> solve_checksum('5 9 2 8\\n9 4 7 3\\n3 8 6 5', even_quotient)
    9
    """
    np_spreadsheet = [[int(c) for c in l.split()] 
                      for l in spreadsheet.splitlines()]
    return sum(row_checksum(row) for row in np_spreadsheet)

In [ ]:
# Run unit tests
doctest.testmod()

In [ ]:
# Final answer
with open('day2.txt') as f:
    spreadsheet = f.read().strip()
    print('Part 1: ', solve_checksum(spreadsheet, row_checksum=minmax_sum))
    print('Part 2: ', solve_checksum(spreadsheet, row_checksum=even_quotient))

## Day 3: Spiral Memory

```
37  36  35  34  33  32  31
38  17  16  15  14  13  30
39  18   5   4   3  12  29
40  19   6   1   2  11  28
41  20   7   8   9  10  27
42  21  22  23  24  25  26
43  44  45  46  47  48  49 ...
```

In [ ]:
def spiral_to_ring_size(spiral_coord):
    return math.ceil(math.sqrt(spiral_coord)) | 1

def ring_size_to_ring(ring_size):
    return ring_size // 2 + 1

def spiral_to_ring(spiral_coord):
    """
    Returns the ring that the coordinate appears in, starting with 1 as the center.
    
    >>> spiral_to_ring(1)
    1
    >>> spiral_to_ring(4)
    2
    >>> spiral_to_ring(20)
    3
    >>> spiral_to_ring(49)
    4
    """
    return ring_size_to_ring(spiral_to_ring_size(spiral_coord))

_right = lambda r: ((r-1, i) for i in range(-r+2, r))
_top = lambda r: ((i, r-1) for i in range(r-2, -r, -1))
_left = lambda r: ((-r+1, i) for i in range(r-2, -r, -1))
_bottom = lambda r: ((i, -r+1) for i in range(-r+2, r))
def ring_to_cartesians(ring):
    """
    Returns the cartesian coordinates for cells in the ring in CCW order,
    starting from the cell just above the bottom-right corner.
    
    >>> list(ring_to_cartesians(1))
    [(0, 0)]
    >>> list(ring_to_cartesians(2))
    [(1, 0), (1, 1), (0, 1), (-1, 1), (-1, 0), (-1, -1), (0, -1), (1, -1)]
    >>> list(ring_to_cartesians(3))
    [(2, -1), (2, 0), (2, 1), (2, 2), (1, 2), (0, 2), (-1, 2), (-2, 2), (-2, 1), (-2, 0), (-2, -1), (-2, -2), (-1, -2), (0, -2), (1, -2), (2, -2)]
    """
    if ring == 1:
        return [(0, 0)]
    return itertools.chain(_right(ring), _top(ring), _left(ring), _bottom(ring))

def spiral_to_cartesian(spiral_coord):
    """
    >>> spiral_to_cartesian(1)
    (0, 0)
    >>> spiral_to_cartesian(9)
    (1, -1)
    >>> spiral_to_cartesian(7)
    (-1, -1)
    >>> spiral_to_cartesian(28)
    (3, 0)
    """
    ring_size = spiral_to_ring_size(spiral_coord)
    ring = ring_size_to_ring(ring_size)
    inner_ring_size = max(ring_size-2, 0) ** 2
    ring_coord = spiral_coord - inner_ring_size - 1
    return nth(ring_to_cartesians(ring), ring_coord)

def solve_shortest_distance(spiral_coord):
    """
    >>> solve_shortest_distance(1)
    0
    >>> solve_shortest_distance(12)
    3
    >>> solve_shortest_distance(23)
    2
    >>> solve_shortest_distance(1024)
    31
    """
    x, y = spiral_to_cartesian(spiral_coord)
    return abs(x) + abs(y)

In [ ]:
spiral_to_cartesian(1)

In [ ]:
# Run unit tests
doctest.testmod()

In [ ]:
# Final answer
print('Part 1: ', solve_shortest_distance(368078))

In [ ]:
# Part 2 will require a total reimplementation...
def generate_spiral():
    ring = 1
    while True:
        for cartesian_coords in ring_to_cartesians(ring):
            yield cartesian_coords
        ring += 1

def solve_shortest_distance2(spiral_coord):
    """
    >>> solve_shortest_distance2(1)
    0
    >>> solve_shortest_distance2(12)
    3
    >>> solve_shortest_distance2(23)
    2
    >>> solve_shortest_distance2(1024)
    31
    """
    (x, y) = nth(generate_spiral(), spiral_coord - 1)
    return abs(x) + abs(y)

In [ ]:
doctest.testmod()

In [ ]:
# Final answer part 1 redux
print('Part 1: ', solve_shortest_distance2(368078))

In [ ]:
# Now, on to part 2
def sum_neighbors(grid, xy):
    x, y = xy
    deltas = [(-1, -1), (-1, 0), (-1, 1), (0, 1), (1, 1), (1, 0), (1, -1), (0, -1)]
    return sum(grid[(x+dx, y+dy)] for (dx, dy) in deltas)

def solve_first_larger_than(seek_value):
    """
    >>> solve_first_larger_than(3)
    4
    >>> solve_first_larger_than(12)
    23
    >>> solve_first_larger_than(23)
    25
    >>> solve_first_larger_than(500)
    747
    """
    grid = defaultdict(int)
    for xy in generate_spiral():
        total = sum_neighbors(grid, xy) or 1
        if total > seek_value:
            return total
        grid[xy] = total

In [ ]:
doctest.testmod()

In [ ]:
# Final answer part 2
print('Part 2: ', solve_first_larger_than(368078))

## Day 4: High-Entropy Passphrases

In [ ]:
identity = lambda x: x
split_whitespace = lambda pp: pp.split(' ')
normalize = lambda t: ''.join(sorted(t))

def is_valid_passphrase(pp, tokenizer=split_whitespace, normalizer=identity):
    """
    >>> is_valid_passphrase("aa bb cc dd ee", split_whitespace)
    True
    >>> is_valid_passphrase("aa bb cc dd aa", split_whitespace)
    False
    >>> is_valid_passphrase("aa bb cc dd aaa", split_whitespace)
    True
    >>> is_valid_passphrase("abcde fghij", split_whitespace, normalize)
    True
    >>> is_valid_passphrase("abcde xyz ecdab", split_whitespace, normalize)
    False
    >>> is_valid_passphrase("a ab abc abd abf abj", split_whitespace, normalize)
    True
    >>> is_valid_passphrase("iiii oiii ooii oooi oooo", split_whitespace, normalize)
    True
    >>> is_valid_passphrase("oiii ioii iioi iiio", split_whitespace, normalize)
    False
    """
    tokens = [normalizer(t) for t in tokenizer(pp)]
    return len(set(tokens)) == len(tokens)

In [ ]:
doctest.testmod()

In [ ]:
# Final answer
with open('day4.txt') as f:
    pps = [l.strip() for l in f]
    print('Part 1: ', sum(is_valid_passphrase(pp) for pp in pps))
    print('Part 2: ', sum(is_valid_passphrase(pp, normalizer=normalize) for pp in pps))

## Day 5: A Maze of Twisty Trampolines, All Alike

In [ ]:
def increment(maze, pos): maze[pos] += 1
def three_or_more(maze, pos): maze[pos] += (1 if maze[pos] < 3 else -1)
    
def solve_maze(maze, pos=0, update=increment):
    """
    >>> solve_maze([0, 3,  0,  1,  -3], update=increment)
    5
    >>> solve_maze([0, 3,  0,  1,  -3], update=three_or_more)
    10
    """
    t = 0
    maze_size = len(maze)
    while pos < maze_size:
        offset = maze[pos]
        update(maze, pos)
        pos += offset
        t += 1
    return t

In [ ]:
doctest.testmod()

In [ ]:
# Final answer
with open('day5.txt') as f:
    maze = [int(l) for l in f]
    %time print('Part 1: ', solve_maze(maze.copy(), update=increment))
    %time print('Part 2: ', solve_maze(maze.copy(), update=three_or_more))

## Day 6: Memory Reallocation

In [ ]:
def _spread(N, start, amount):
    """
    >>> print(_spread(4, 0, 7))
    [2 2 2 1]
    >>> print(_spread(4, 3, 7))
    [2 2 1 2]
    >>> print(_spread(5, 0, 17))
    [4 4 3 3 3]
    """
    a = np.full(N, amount // N)
    a[:(amount % N)] += 1
    return np.roll(a, start)

def _redistribute(banks, pos):
    """
    >>> print(_redistribute(np.array([0, 2, 7, 0]), 2))
    [2 4 1 2]
    >>> print(_redistribute(np.array([2, 4, 1, 2]), 1))
    [3 1 2 3]
    >>> print(_redistribute(np.array([0, 2, 7, 1]), 3))
    [1 2 7 0]
    """
    N = len(banks)
    blocks = banks[pos]
    return (banks 
            - np_impulse(N, pos, blocks) 
            + _spread(N, pos + 1, blocks))
    
def reallocate(banks):
    """
    >>> banks = np.array([0, 2, 7, 0])
    >>> reallocate((0, 2, 7, 0))
    (2, 4, 1, 2)
    >>> reallocate((2, 4, 1, 2))
    (3, 1, 2, 3)
    """
    return tuple(_redistribute(np.array(banks), np.argmax(banks)))

def solve_reallocate_cycles(banks):
    """
    >>> banks = np.array([0, 2, 7, 0])
    >>> solve_reallocate_cycles((0, 2, 7, 0))
    (5, 4)
    """
    seen = {}
    for time in itertools.count(0):
        if banks in seen:
            return time, time - seen[banks]
        seen[banks] = time
        banks = reallocate(banks)

In [ ]:
doctest.testmod(verbose=True)

In [ ]:
# Final answer
with open('day6.txt') as f:
    banks = tuple(int(b) for b in f.read().strip().split('\t'))
    %time (time, cycles) = solve_reallocate_cycles(banks)
    print('Part 1: ', time)
    print('Part 2: ', cycles)

## Day 7: Recursive Circus

In [153]:
node_re = re.compile(r"([a-z]+) \(([\-0-9]+)\)(?: -> ([\w, ]+))?")

Node = namedtuple('Node', ['id', 'weight', 'children'])
Tree = namedtuple('Tree', ['root', 'nodes'])

def parse_node(text):
    """
    >>> parse_node("ktlj (57)")
    Node(id='ktlj', weight=57, children=[])
    >>> parse_node("xyz (-7) -> abc, def, ghi")
    Node(id='xyz', weight=-7, children=['abc', 'def', 'ghi'])
    """
    node_id, weight, children = node_re.match(text).groups()
    weight = int(weight)
    children = [c.strip() for c in children.split(',')] if children else []
    return Node(node_id, weight, children)

def parse_tree(text):
    """
    >>> tree = parse_tree("ktlj (57)\\nxyz (-7) -> ktlj")
    >>> tree.root
    'xyz'
    """
    nodes = {}
    for line in text.splitlines():
        line = line.strip()
        if not line: continue
        node = parse_node(line)
        nodes[node.id] = node
    root = get_root(nodes)
    return Tree(root, nodes)


def tree_weight(tree, node_id=None):
    """
    >>> t = Tree('xyz', {'xyz': Node('xyz', 10, ['abc']), 'abc': Node('abc', -2, [])})
    >>> tree_weight(t, 'xyz')
    8
    >>> tree_weight(t, 'abc')
    -2
    >>> tree_weight(t)
    8
    """
    node_id = node_id or tree.root
    node = tree.nodes[node_id]
    return node.weight + sum(tree_weight(tree, c) for c in node.children)

def verify_tree(tree, node_id=None):
    node_id = node_id or tree.root
    node = tree.nodes[node_id]
    if not node.children:
        return node.weight
    ws = [verify_tree(tree, c) for c in node.children]
    if len(set(ws)) != 1:
        bad_idx = list((ws[i] == ws[i-1]) or (ws[i] == ws[i-2]) 
                       for i in range(len(ws))).index(False)
        bad_node = tree.nodes[node.children[bad_idx]]
        bad_weight = ws[bad_idx]
        good_weight = ws[bad_idx - 1]
        correction = (good_weight - bad_weight)
        raise Exception('Failed verification at node <{}> due to child <{}>'.format(node_id, 
                                                                                    bad_node.id),
                        node.children[bad_idx],
                        bad_node.weight + correction)
    else:
        return node.weight + sum(ws)
        

def get_root(nodes):
    """
    >>> get_root({'xyz': Node('xyz', 1, children=['abc']), 'abc': Node('abc', 2, [])})
    'xyz'
    """
    all_nodes = set(nodes.keys())
    child_nodes = set(
        itertools.chain.from_iterable(n.children for n in nodes.values()))
    return (all_nodes - child_nodes).pop()
        
def solve_bottom_program(tree):
    return tree.root

In [154]:
doctest.testmod(verbose=True)

test_tree = parse_tree("""
pbga (66)
xhth (57)
ebii (61)
havc (66)
ktlj (57)
fwft (72) -> ktlj, cntj, xhth
qoyq (66)
padx (45) -> pbga, havc, qoyq
tknk (41) -> ugml, padx, fwft
jptl (61)
ugml (68) -> gyxo, ebii, jptl
gyxo (61)
cntj (57)
""")

print('Root is:', test_tree.root)
assert test_tree.root == 'tknk'

try:
    verify_tree(test_tree)
except Exception as e:
    msg, bad_node, corrected_weight = e.args
    assert corrected_weight == 60

Trying:
    get_root({'xyz': Node('xyz', 1, children=['abc']), 'abc': Node('abc', 2, [])})
Expecting:
    'xyz'
ok
Trying:
    parse_node("ktlj (57)")
Expecting:
    Node(id='ktlj', weight=57, children=[])
ok
Trying:
    parse_node("xyz (-7) -> abc, def, ghi")
Expecting:
    Node(id='xyz', weight=-7, children=['abc', 'def', 'ghi'])
ok
Trying:
    tree = parse_tree("ktlj (57)\nxyz (-7) -> ktlj")
Expecting nothing
ok
Trying:
    tree.root
Expecting:
    'xyz'
ok
Trying:
    t = Tree('xyz', {'xyz': Node('xyz', 10, ['abc']), 'abc': Node('abc', -2, [])})
Expecting nothing
ok
Trying:
    tree_weight(t, 'xyz')
Expecting:
    8
ok
Trying:
    tree_weight(t, 'abc')
Expecting:
    -2
ok
Trying:
    tree_weight(t)
Expecting:
    8
ok
13 items had no tests:
    __main__
    __main__.Node
    __main__.Node.children
    __main__.Node.id
    __main__.Node.weight
    __main__.Tree
    __main__.Tree.nodes
    __main__.Tree.root
    __main__.build_tree
    __main__.np_impulse
    __main__.nth
    __main

In [158]:
# Final answer
with open('day7.txt') as f:
    tree = parse_tree(f.read())
    print('Part 1: root node it', tree.root)
    try:
        verify_tree(tree)
    except Exception as e:
        print('Part 2: corrected weight is', e.args[2])

Part 1: root node it mwzaxaj
Part 2: corrected weight is 1219
